In [ ]:
import networkx as nx

from math import sqrt

from bokeh.plotting import show, figure
from bokeh.io import output_notebook
from bokeh.models import HoverTool

from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes,from_networkx
from bokeh.palettes import Spectral4

In [2]:
network = nx.read_gml('subgraph.gml')

In [4]:
import numpy as np
import operator

class node:
    def __init__(self, name, start, stop):
        self.name = name
        self.start = start
        self.stop = stop
        
    def __repr__(self):
        return repr((self.name, self.start, self.stop))

In [5]:
node_list = []

for name in network:
    position = name.split(';')[3]
    position = position.split('=')[1]
    position = (position.split(',')[0],position.split(',')[1])
    
    node_list.append(node(name, position[0], position[1]))

sorted(node_list, key = lambda node: node.start)


[('719.1;len=100;read_position=0;template_position=765,853', '765', '853'),
 ('1758.2;len=100;read_position=0;template_position=769,857', '769', '857'),
 ('283.2|1998.2|175.2|1998.2|283.2|175.2;len=100;read_position=0;template_position=773,863', '773', '863'),
 ('908.1;len=100;read_position=0;template_position=775,873', '775', '873'),
 ('2167.1;len=100;read_position=0;template_position=782,881', '782', '881'),
 ('2801.2;len=100;read_position=0;template_position=788,887', '788', '887'),
 ('2499.2;len=100;read_position=0;template_position=789,888', '789', '888'),
 ('647.1;len=100;read_position=0;template_position=795,894', '795', '894'),
 ('607.1;len=100;read_position=0;template_position=799,898', '799', '898'),
 ('2922.1;len=100;read_position=0;template_position=801,900', '801', '900'),
 ('2533.2|1751.2|1788.2;len=100;read_position=0;template_position=803,902', '803', '902'),
 ('351.2;len=100;read_position=0;template_position=804,903', '804', '903'),
 ('233.2;len=100;read_position=0;tem

In [8]:
for name in network:
    print(name)

1298.1
1479.2
1735.1
1648.1
417.2
1210.1
2917.2
351.1
3079.1
886.1
1446.1
247.1
908.1
2801.2
1758.2
2499.2
2922.1
2167.1
1990.1
351.2
233.2
2423.1
3300.2
233.1
1200.2
1200.1
2533.2|1751.2|1788.2
647.1
3057.1
607.1
283.2|1998.2|175.2|1998.2|283.2|175.2
719.1
840.2|3067.2|647.2


In [7]:
flag = False

for node in network.nodes():
    if 'read_position' in node:
        flag = True
        break
        
if flag == True:
    mapping = {}
    for node in network.nodes():
        mapping[node] = node.split(';')[0]
    
    network = nx.relabel_nodes(network, mapping) 

NameError: name 'nodePosition' is not defined

In [10]:
#layout = nodePosition
layout = nx.spring_layout(network, k=1.1/sqrt(network.number_of_nodes()),iterations=100, )

from bokeh.models import ColumnDataSource

nodes, nodes_coordinates = zip(*sorted(layout.items()))
nodes_xs, nodes_ys = list(zip(*nodes_coordinates))
nodes_source = ColumnDataSource(dict(x=nodes_xs, y=nodes_ys,
                                     name=nodes))

#from bokeh.models import ColumnDataSource, LabelSet

#x,y=zip(*layout.values())
#names = layout.keys()

#source = ColumnDataSource({'x':x,'y':y,'name':names})
#labels = LabelSet(x='x', y='y', text='Seq', source=source)

In [11]:
layout

{'1200.1;len=100;read_position=0;template_position=822,921': array([-0.20359738, -0.2133139 ]),
 '1200.2;len=100;read_position=0;template_position=884,972': array([-0.80578656, -0.19103055]),
 '1210.1;len=100;read_position=0;template_position=828,927': array([ 0.34669261,  0.2531573 ]),
 '1298.1;len=100;read_position=0;template_position=812,911': array([-0.05601333, -0.21348703]),
 '1446.1;len=100;read_position=0;template_position=845,943': array([-0.27719918, -0.20931979]),
 '1479.2;len=100;read_position=0;template_position=834,933': array([-0.16432062, -0.21602797]),
 '1648.1;len=100;read_position=0;template_position=905,993': array([-1.        , -0.23628883]),
 '1735.1;len=100;read_position=0;template_position=904,992': array([-0.88365463, -0.21893091]),
 '1758.2;len=100;read_position=0;template_position=769,857': array([ 0.13324219, -0.03672116]),
 '1990.1;len=100;read_position=0;template_position=895,982': array([ 0.72421275,  0.46446718]),
 '2167.1;len=100;read_position=0;templat

In [12]:
hover = HoverTool(tooltips=[('name', '@name'), ('id', '$index')])
plot = figure(plot_width=1500, plot_height=1000,
              tools=['tap', hover, 'box_zoom', 'reset'])
r_circles = plot.circle('x', 'y', source=nodes_source, size=10,
                        color='blue', level = 'overlay')

In [13]:
output_file("interactive_graphs.html"); show(plot)

In [14]:
def get_edges_specs(_network, _layout):
    d = dict(xs=[], ys=[], alphas=[])
    weights = [d['overlap'] for u, v, d in _network.edges(data=True)]
    max_weight = max(weights)
    calc_alpha = lambda h: 0.1 + 0.6 * (h / max_weight)

    # example: { ..., ('user47', 'da_bjoerni', {'weight': 3}), ... }
    for u, v, data in _network.edges(data=True):
        d['xs'].append([_layout[u][0], _layout[v][0]])
        d['ys'].append([_layout[u][1], _layout[v][1]])
        d['alphas'].append(calc_alpha(data['overlap']))
    return d

In [15]:
lines_source = ColumnDataSource(get_edges_specs(network, layout))

r_lines = plot.multi_line('xs', 'ys', line_width=1.5,
                          alpha='alphas', color='navy',
                          source=lines_source)
show(plot)

In [22]:
plot = Plot(plot_width=1000, plot_height=1000,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))

plot.title.text = "Subgraph plot"

graph_renderer = from_networkx(network, nx.spring_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=5)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)

plot.add_tools(HoverTool(tooltips=[('name', '@name'), ('id', '$index')]), TapTool(), BoxSelectTool())

output_file("interactive_graphs.html")

show(plot)


In [59]:
x,y=zip(*layout.values())
names = layout.keys()
print("x: {}, y: {}, names: {}".format(x, y, names))


SyntaxError: can't use starred expression here (<ipython-input-59-d8938ec94f1f>, line 2)